In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
#Read in the csv file and turn the ContractDate column to datetime object
commish = pd.read_csv('Bags Medical Devices Company- Commision Detail.csv', 
                      parse_dates=['ContractDate'])

In [ ]:
commish.describe()

In [ ]:
commish.head()

In [ ]:
commish.tail()

In [ ]:
commish.columns

In [20]:
commish.dtypes

ContractId                      int64
ContractYear                    int64
ContractDate           datetime64[ns]
PrimarySalesRep                object
SecondarySalesRep              object
ContractAmount                  int64
PrimaryCommission             float64
SecondaryCommission           float64
TotalCommission               float64
dtype: object

In [21]:
def tweak_commish(df):
    return(df
          .loc[:, ['ContractId', 'ContractYear', 'ContractDate', 
                   'PrimarySalesRep', 'SecondarySalesRep', 'ContractAmount']]
          )

def commission(df):
    #Function to compute commissions and Total Commission
    def primary(df_):
        #Function to compute primary commissions for SR
        return(df_
              .assign(PrimaryCommission = lambda x: x['ContractAmount'] * 0.1)
              )
    
    def secondary(df_):
        #Function to compute secondary commissions for SR
        if ['SecondarySalesRep' == 'No secondary sales rep']:
            df_.assign(SecondaryCommission = 0)
        else:
            df_.assign(SecondaryCommission = lambda x: x['ContractAmount'] * 0.05)
    
    def total(df_):
        #Function to create Total commission for contract
        return(df_
              .assign(TotalCommission = PrimaryCommission + SecondaryCommission)
              )
    
    return(df
          .assign(PrimaryCommission = primary,
                  SecondaryCommission = secondary,
                 TotalCommission = total))
   

In [34]:
#ChatGPT Code Fix
def commission(df):
    # Function to compute commissions and Total Commission
    def primary(df_):
        # Function to compute primary commissions for SR
        return df_.assign(PrimaryCommission=lambda x: x['ContractAmount'] * 0.1)

    def secondary(df_):
        # Function to compute secondary commissions for SR
        df_['SecondaryCommission'] = 0
        df_.loc[df_['SecondarySalesRep'] != 'No secondary sales rep', 'SecondaryCommission'] = df_['ContractAmount'] * 0.05
        return df_

    def total(df_):
        # Function to create Total commission for contract
        return df_.assign(TotalCommission=lambda x: x['PrimaryCommission'] + x['SecondaryCommission'])

    return (
        df.pipe(primary)
          .pipe(secondary)
          .pipe(total)
    )

In [23]:
commish = tweak_commish(commish)

commish

,ContractId,ContractYear,ContractDate,PrimarySalesRep,SecondarySalesRep,ContractAmount
0,1,2008,2008-01-04,Inez Sanchez,Fred Jones,37419
1,2,2008,2008-01-07,Brady Marshall,No secondary sales rep,80152
2,3,2008,2008-01-11,Brady Marshall,Laura Barber,58643
3,4,2008,2008-01-12,Mitchel Bogart,No secondary sales rep,22380
4,5,2008,2008-01-12,Fred Jones,Kyle Freeman,50651
...,...,...,...,...,...,...
564,565,2011,2011-12-18,Laura Barber,No secondary sales rep,22003
565,566,2011,2011-12-19,Mitchel Bogart,Amanda Smith,33771
566,567,2011,2011-12-24,Kyle Freeman,Sandy McKnight,16803
567,568,2011,2011-12-26,Mitchel Bogart,Amanda Smith,18603


In [35]:
commish = commission(commish)

commish

,ContractId,ContractYear,ContractDate,PrimarySalesRep,SecondarySalesRep,ContractAmount,PrimaryCommission,SecondaryCommission,TotalCommission
0,1,2008,2008-01-04,Inez Sanchez,Fred Jones,37419,3741.9,1870.95,5612.85
1,2,2008,2008-01-07,Brady Marshall,No secondary sales rep,80152,8015.2,0.00,8015.20
2,3,2008,2008-01-11,Brady Marshall,Laura Barber,58643,5864.3,2932.15,8796.45
3,4,2008,2008-01-12,Mitchel Bogart,No secondary sales rep,22380,2238.0,0.00,2238.00
4,5,2008,2008-01-12,Fred Jones,Kyle Freeman,50651,5065.1,2532.55,7597.65
...,...,...,...,...,...,...,...,...,...
564,565,2011,2011-12-18,Laura Barber,No secondary sales rep,22003,2200.3,0.00,2200.30
565,566,2011,2011-12-19,Mitchel Bogart,Amanda Smith,33771,3377.1,1688.55,5065.65
566,567,2011,2011-12-24,Kyle Freeman,Sandy McKnight,16803,1680.3,840.15,2520.45
567,568,2011,2011-12-26,Mitchel Bogart,Amanda Smith,18603,1860.3,930.15,2790.45


In [38]:
commish['PrimarySalesRep'].value_counts()

Mitchel Bogart      59
Fred Jones          56
Willie Johnson      50
Laura Barber        50
Jim Jackson         43
Inez Sanchez        37
Amanda Smith        36
Jason Willis        35
Kelly Thomas        35
Kyle Freeman        34
Brady Marshall      32
Sandy McKnight      31
Victoria Beckham    29
Mark Esposito       21
Ryan Leaf           21
Name: PrimarySalesRep, dtype: int64

In [37]:
commish['SecondarySalesRep'].value_counts()

No secondary sales rep    293
Amanda Smith               36
Laura Barber               27
Sandy McKnight             25
Kyle Freeman               24
Brady Marshall             24
Jason Willis               22
Willie Johnson             19
Victoria Beckham           17
Fred Jones                 15
Mark Esposito              15
Ryan Leaf                  13
Kelly Thomas               12
Inez Sanchez               11
Jim Jackson                10
Mitchel Bogart              6
Name: SecondarySalesRep, dtype: int64